## 移动设备发展
![image](memory1_page1.png)

- 手机运行内存（RAM）其实相当于我们的 PC 中的内存，是手机中作为 App 运行过程中临时性数据暂时存储的内存介质。考虑到体积和功耗，手机不使用 PC 的 DDR 内存，采用的是 LPDDR RAM，全称是“低功耗双倍数据速率内存”，其中 LP 就是 "Lower Power" 低功耗的意思。
- 以 LPDDR4 为例，带宽 = 时钟频率 × 内存总线位数÷ 8，即 1600 × 64 ÷ 8 = 12.8GB/s，因为是 DDR 内存是双倍速率，所以最后的带宽是 12.8 × 2 = 25.6GB/s。

- 目前市面上的手机，主流的运行内存有 LPDDR3、LPDDRLPDDR4 以及 LPDDR4X。LPDDR4 的性能要比 LPDDR3 高出一倍，而 LPDDR4X 相比 LPDDR4 工作电压更低，所以也比 LPDDR4 省电 20%～40%。

![image](memory1_page2.png)

---

## 内存问题
![image](memory1_page3.png)

### 内存造成的问题
- 异常
  - 异常包括 OOM、内存分配失败这些崩溃，也包括因为整体内存不足导致应用被杀死、设备重启等问题。

- 卡顿:
  - Java 内存不足会导致频繁 GC，这个问题在 Dalvik 虚拟机会更加明显。
  - 而 ART 虚拟机在内存管理跟回收策略上都做大量优化，内存分配和 GC 效率相比提升了 5～10 倍。
  - 如果想具体测试 GC 的性能，例如暂停挂起时间、总耗时、GC 吞吐量，可以通过发送 **SIGQUIT 信号获得 ANR 日志**。
  - 另外还可以使用 systrace 来观察 GC 的性能耗时
  
  ```
  // 命令
  adb shell kill -S QUIT PID
  adb pull /data/anr/traces.txt
  
  // 输出
  sticky concurrent mark sweep paused:Sum: 5.491ms 99% C.I. 1.464ms-2.133ms Avg: 1.830ms Max: 2.133ms // GC 暂停时间
  Total time spent in GC: 502.251ms     // GC 总耗时
  Mean GC size throughput: 92MB/s       // GC 吞吐量
  Mean GC object throughput: 1.54702e+06 objects/s 
  ```
  
  - 除了频繁 GC 造成卡顿之外，物理内存不足时系统会触发 low memory killer 机制，系统负载过高是造成卡顿的另外一个原因。
  
#### 两个误区
- 误区一：内存占用越少越好
  - VSS、PSS、Java 堆内存不足都可能会引起异常和卡顿
  - 当系统内存充足的时候，我们可以多用一些获得更好的性能。当系统内存不足的时候，希望可以做到 "用时分配，及时释放"。当系统内存出现压力时，能够迅速释放各种缓存来减少系统压力。

  ![image](memory1_page4.png)
  
> 回归下 Android Bitmap 内存分配的变化
  - 在 Android 3.0 之前，Bitmap 对象放在 Java 堆，而像素数据是放在 Native 内存中。如果不手动调用 recycle，Bitmap Native 内存的回收完全依赖 finalize 函数回调，这个时机不太可控。
  - Android 3.0 ～ Android 7.0 将 Bitmap 对象和像素数据统一放到 Java 堆中，这样就算我们不调用 recycle，Bitmap 内存也会随着对象一起被回收。即使是最新的华为 Mate 20，最大的 Java 堆限制也才到 512MB。Bitmap 放到 Java 堆的另外一个问题会引起大量的 GC，对系统内存也没有完全利用起来。
  - 有没有一种实现，可以将 Bitmap 内存放到 Native 中，也可以做到和对象一起快速释放，同时 GC 的时候也能考虑这些内存防止被滥用？NativeAllocationRegistry 可以一次满足你这三个要求，Android 8.0 正是使用这个辅助回收 Native 内存的机制，来实现像素数据放到 Native 内存中。Android 8.0 还新增了硬件位图 Hardware Bitmap，它可以减少图片内存并提升绘制效率。

- 误区二：Native 内存不用管
  - 当系统物理内存不足时，lmk 开始杀进程，从后台、桌面、服务、前台，直到手机重启。
  - 在 Android 8.0 以后应用保活变得困难很多，但依然有一些方法可以突破。
  
  ![image](memory1_page5.png)

---

### 图片内存放到 native 中
- Fresco 图片库在 Dalvik 会把图片放到 Native 内存中。事实上在 Android 5.0 ～ Android 7.0，也能做到相同的效果，只是流程相对复杂一些

#### 步骤一
- 通过直接调用 libandroid_runtime.so 中 Bitmap 的构造函数，可以得到一张空的 Bitmap 对象，而它的内存是放到 Native 堆中。但是不同 Android 版本的实现有那么一点差异，这里都需要适配。

#### 步骤二
- 通过系统的方法创建一张普通的 Java Bitmap。

#### 步骤三
- 将 Java Bitmap 的内容绘制到之前申请的空的 Native Bitmap 中。

#### 步骤四
- 将申请的 Java Bitmap 释放，实现图片内存的 "偷龙转凤"。

```
// 步骤一：申请一张空的 Native Bitmap
Bitmap nativeBitmap = nativeCreateBitmap(dstWidth, dstHeight, nativeConfig, 22);

// 步骤二：申请一张普通的 Java Bitmap
Bitmap srcBitmap = BitmapFactory.decodeResource(res, id);

// 步骤三：使用 Java Bitmap 将内容绘制到 Native Bitmap 中
mNativeCanvas.setBitmap(nativeBitmap);
mNativeCanvas.drawBitmap(srcBitmap, mSrcRect, mDstRect, mPaint);

// 步骤四：释放 Java Bitmap 内存
srcBitmap.recycle();
srcBitmap = null；
```

> 虽然最终图片的内存的确是放到 Native 中了，不过这个 "黑科技" 有两个主要问题，一个是兼容性问题，另外一个是频繁申请释放 Java Bitmap 容易导致内存抖动。

---

## 测量方法
- 对于系统内存和应用内存的使用情况，你可以参考 Android Developer 中 [调查 RAM 使用情况](https://developer.android.com/studio/profile/investigate-ram?hl=zh-cn)

```
adb shell dumpsys meminfo <package_name|pid> [-d]
```

### 1. Java 内存分配
- 有些时候我们希望跟踪 Java 堆内存的使用情况，这个时候最常用的有 Allocation Tracker 和 MAT 这两个工具。

> [Android 内存申请分析](https://mp.weixin.qq.com/s/b_lFfL1mDrNVKj_VAcA2ZA?) 文章里提到的 Allocation Tracker 的三个缺点
  - 获取的信息过于分散，中间夹杂着不少其他的信息，很多信息不是应用申请的，可能需要进行不少查找才能定位到具体的问题。
  - 跟 Traceview 一样，无法做到自动化分析，每次都需要开发者手工开始 / 结束，这对于某些问题的分析可能会造成不便，而且对于批量分析来说也比较困难。
  - 虽然在 Allocation Tracking 的时候，不会对手机本身的运行造成过多的性能影响，但是在停止的时候，直到把数据 dump 出来之前，经常会把手机完全卡死，如果时间过长甚至会直接 ANR。

- 因此我们希望可以做到脱离 Android Studio，实现一个自定义的 "Allocation Tracker"，实现对象内存的自动化分析。通过这个工具可以获取所有对象的申请信息（大小、类型、堆栈等），可以找到一段时间内那些对象占用了大量的内存。
- 这个方法需要考虑的兼容性问题会比较多，在 Dalvik 和 ART 中，Allocation Tracker 的处理流程差异就非常大。
- Allocation Tacker 的开启方式:

```
// dalvik
bool dvmEnableAllocTracker()

// art
void setAllocTrackingEnabled()
```

### 2. Native 内存分配
- Google 之前将 Valgrind 弃用，建议我们使用 Chromium 的 [AddressSanitize](http://source.android.com/devices/tech/debug/asan.html)。遵循 "谁最痛，谁最需要，谁优化"，所以 Chromium 出品了一大堆 Native 相关的工具。
- Android 之前对 AddressSanitize 支持的不太好，需要 root 和一大堆的操作，在 Android 8.0 之后，我们可以根据这个[指南](http://github.com/google/sanitizers/wiki/AddressSanitizerOnAndroid)来使用 AddressSanitize。
- 目前 AddressSanitize 内存泄漏检测只支持 x86_64 Linux 和 OS X 系统。

- 有没有类似 Allocation Tracker 那样的 Native 内存分配工具呢？这方面 Android 目前的支持还不是太好，但 Android Developer 近来也补充了一些相关的文档，你可以参考[《调试本地内存使用》](https://source.android.com/devices/tech/debug/native-memory)。

#### Native 内存的问题
- 有两种方法，分别是 Malloc 调试和 Malloc 钩子。

> [Malloc 调试](https://android.googlesource.com/platform/bionic/+/master/libc/malloc_debug/README.md)
- 可以帮助我们去调试 Native 内存的一些使用问题，例如堆破坏、内存泄漏、非法地址等。
- Android 8.0 之后支持在非 root 的设备做 Native 内存调试，不过跟 AddressSanitize 一样，需要通过 [wrap.sh](http://developer.android.com/ndk/guides/wrap-script.html) 做包装。
```
adb shell setprop wrap.<APP> '"LIBC_DEBUG_MALLOC_OPTIONS=backtrace logwrapper"'
```

> [Malloc 钩子](https://android.googlesource.com/platform/bionic/+/master/libc/malloc_hooks/README.md)
- 在 Android P 之后，Android 的 libc 支持拦截在程序执行期间发生的所有分配 / 释放调用，这样我们就可以构建出自定义的内存检测工具。
```
adb shell setprop wrap.<APP> '"LIBC_HOOKS_ENABLE=1"'
```

- 在使用 "Malloc 调试" 时，感觉整个 App 都会变卡，有时候还会产生 ANR。

---

## 总结
- LPDDR5 将在明年进入量产阶段，移动内存一直向着更大容量、更低功耗、更高带宽的方向发展。伴随内存的发展，内存优化的挑战和解决方案也不断变化。而内存优化又是性能优化重要的一部分。
- 在设计方案的时候，还需要考虑要使用多少的内存，应该怎么去管理这些内存。在需求完成之后，我们也应该去回归需求的内存情况，是否存在使用不当的地方，是否出现内存泄漏。

---

## 额外补充
### 工具介绍
- Memory Monitor：跟踪整个 app 的内存变化情况
- Heap Viewer：查看当前内存快照，便于对比分析哪些对象有可能发生了泄漏
- Allocation Tracker：追踪内存对象的来源

### Traceview Walkthrough
- 通过 Android Studio 打开里面的 Android Device Monitor，切换到 DDMS 窗口，点击左边栏上面想要跟踪的进程，再点击上面的 Start Method Tracing 的按钮，即可启动跟踪。
- 启动跟踪之后，再操控 app，做一些你想要跟踪的事件，例如滑动 listview，点击某些视图进入另外一个页面等等。操作完之后，回到 Android Device Monitor，再次点击 Method Tracing 的按钮停止跟踪。此时工具会为刚才的操作生成 TraceView 的详细视图。

---

## 课后作业
- 本期的 Sample 就提供了一个自定义的 Allocation Tracker 实现的示例，目前已经兼容到 Android 8.1。用它练习实现自动化的内存分析，有哪些对象占用了大量内存，以及它们是如何导致 GC 等。

---